# Loading the Libraries

In [1]:
import pandas as pd
import numpy as np
import string
import nltk
from textblob import TextBlob
from geotext import GeoText
from sklearn import preprocessing
import tweepy
from tweepy import API
import datetime
import urllib
import face_recognition
from difflib import SequenceMatcher
import gender_guesser.detector as gender
import time
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

## Initiating Connection to Twitter API

In [2]:
consumer_key="I8ZQyd1mWDFdq6yPC4EaMhuUJ"
consumer_secret="s4d9EL27YFzPikI6BZm96t7CgZvH07r3YH6sBClA2o5bI5LhGt"

access_token="1000407227130368000-08UrKjmG1TPrLTKqyCsl8VJ9lIEfUp"
access_token_secret="brBbDMEEZxuNO2cQ9If1uGzEAeaSHWg83lMYVX2lLdKZj"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

## Loading the Dataset file

In [3]:
# load the dataset
df=pd.read_csv('150425104337_nepal_earthquake_20150425_vol-1.json.csv',sep=',', header=None, index_col=False) 
# drop the column user_id
df = df.drop([1], axis=1)
# drop the first row 
df = df.drop([0])
# name the column
df.columns = ["tweet_id"]  
df = df.reset_index(drop=True)

## Extracting Only the Text

In [ ]:
# initialize the lists
text=[]
deleted_ids=[]
requests=0
time_rate = datetime.datetime.now()

for i in range(len(df)):
    
    if i % 100 == 0:
        
        print(i)
    
    requests = requests + 1
    
    check = datetime.datetime.now()
    
    if (check - time_rate).seconds > 900 and requests < 850:
        
        time_rate = check
        
        requests = 1
    
    if (check - time_rate).seconds < 900 and requests == 850:
        
        time.sleep(900 - (check - time_rate).seconds)
        
        time_rate = check
        
        requests = 1
    
    try:
    
        obj = api.get_status(df.tweet_id[i][1:-1], tweet_mode="extended")
    
    except tweepy.TweepError:  # in case a tweet is deleted by its user an error will appear
                                # I am creating an exception rule so that my code wont crush
        deleted_ids.append(df.tweet_id[i])  # I am writing down the location of the missing ids so that I will delete the missing tweets
        
        continue       
    
    try:
    
        text.append(obj.full_text)
        
    except:
        
        text.append('NaN')

## Extracting data from Twitter API

In [ ]:
# initialize the lists

favourite_count = []

tweet_age = []

retweet_count = []

profile_age = []

user_description = []

followers = []

friends = []

following = []

geo_enabled = []

extended_profile = []

user_location = []

name = []

profile_image = []

statuses_count = []

user_time_zone = []

verified = []

is_translator_enabled = []

listed_count = []

user_id = []

protected = [] 

screen_name = []

time_now = datetime.datetime.now()

time_rate = time_now

image_exists = []

gender_binary = []

name_male = []

user_lang = []

deleted_tweets = []

deleted_ids = []

notifications = []

text=[]

default_profile_picture = []

requests = 0

error_objects = []

rumor = []

event=[]

# define iterative process that will extract information from Twitter
for i in range(len(df)):
    
    # mechanism to control the algorithm from exceeding Twitter's rate limit of
    # of 900 requests per 15 minutes
    requests = requests + 1
    
    check = datetime.datetime.now()
    
    if (check - time_rate).seconds > 900 and requests < 900:
        
        time_rate = check
        
        requests = 1
    
    if (check - time_rate).seconds < 900 and requests == 900:
        
        time.sleep(900 - (check - time_rate).seconds)
        
        time_rate = check
        
        requests = 1
    
    if i % 100 == 0:
        
        print(i)
    
    try:
        # claim the Tweepy object of the tweet id
        obj = api.get_status(df.tweet_id[i], tweet_mode="extended")
    
    except tweepy.TweepError:  # in case a tweet is deleted by its user an error will appear
                                # I am creating an exception rule so that my code wont crush
        deleted_ids.append(df.tweet_id[i])  # I am writing down the location of the missing ids so that I will delete the missing tweets
        
        continue
        
    rumor.append(df.rumor[i])
    
    event.append(df.event[i])
    
    try:
    
        favourite_count.append(obj.favorite_count) # I count how many users have favoured this tweet
        
    except:
        
        favourite_count.append(0)
        
        error_objects.append(obj)
        
    try:
    
        creation_time_tweet = obj.created_at  # I extract the creation time of the tweet
    
        time_difference = time_now-creation_time_tweet # I count how much time has passed since then
    
        tweet_age.append(time_difference.seconds) # I turn the difference into seconds
        
    except:
        
        tweet_age.append(0)
        
        error_objects.append(obj)
        
    try:
    
        retweet_count.append(obj.retweet_count) # I count the number of re-tweets
        
    except:
        
        retweet_count.append(0)
        
        error_objects.append(obj)
        
    try:
    
        user_creation = obj.user.created_at  # I extract the creation time of the user's account
    
        user_time_difference = time_now-user_creation # I calculate how old is the user's account given the time now
    
        profile_age.append(user_time_difference.days) # I store the time in days
        
    except:
        
        profile_age.append(0)
        
        error_objects.append(obj)
        
    try:
    
        description = obj.user.description # I check if the user has a description for his/her profile or not
    
        if description == '' :
        
            user_description.append(0)
        
        else:
        
            user_description.append(1)
        
    except:
        
        user_description.append(0)
        
        error_objects.append(obj)
        
    try:
        
        text.append(obj.full_text)
        
    except:
        
        text.append.append(0)
        
        error_objects.append(obj)
        
    try:
        
        followers.append(obj.user.followers_count) # I count the number of followers of the user
        
    except:
        
        followers.append(0)
        
        error_objects.append(obj)
        
    try:
    
        following.append(obj.user.friends_count)  # I count the number of the account that the user is following
        
    except:
        
        following.append(0)
        
        error_objects.append(obj)
        
    try:
    
        geo = obj.user.geo_enabled  # I check if the user has enabled the geo_location system of Twitter
    
        if geo is True:
        
            geo_enabled.append(1) 
        
        else:
        
            geo_enabled.append(0)
            
    except:
        
        geo_enabled.append(0)
        
        error_objects.append(obj)
        
    try:
    
        extended = obj.user.has_extended_profile # I check if the user has enabled an extended Twitter profile or not
    
        if extended is True:
        
            extended_profile.append(1) 
        
        else:
        
            extended_profile.append(0)
            
    except:
        
        extended_profile.append(0)
        
        error_objects.append(obj)
        
    try:
        
        location = obj.user.location  # I check if the user has declared a location or not
    
        if location == 'NONE':
        
            user_location.append(0)
        
        else:
        
            user_location.append(1)
            
    except:
        
        user_location.append(0)
        
        error_objects.append(obj)
    
    try:
    
        statuses_count.append(obj.user.statuses_count) # I count the number of statuses that the user has published
        
    except:
        
        statuses_count.append(0)
        
        error_objects.append(obj)
        
    try:
    
        user_notification = obj.user.notifications
    
        if user_notification is True:
        
            notifications.append(1)
        
        else:
        
            notifications.append(0)
            
    except:
        
        notifications.append(0)
        
        error_objects.append(obj)
        
    try:
    
        time_zone = obj.user.time_zone # I check if the user has declared a timezone
    
        if time_zone == 'None':
        
            user_time_zone.append(0)
        
        else:
        
            user_time_zone.append(1)
            
    except:
        
        user_time_zone.append(0)
        
        error_objects.append(obj)
        
    try:
        
        verification = obj.user.verified # I check if the user has verified his/her account or not
    
        if verification is True:
        
            verified.append(1)
        
        else:
        
            verified.append(0)
            
    except:
        
        verified.append(0)
        
        error_objects.append(obj)
        
    try:
    
        pic_default = obj.user.default_profile_image # I check if the user has verified his/her account or not
    
        if pic_default is True:
        
            default_profile_picture.append(1)
        
        else:
        
            default_profile_picture.append(0)
            
    except:
        
        default_profile_picture.append(0)
        
        error_objects.append(obj)
        
    try:
    
        translation = obj.user.is_translation_enabled  # I check if the user has enabled the Twitter's translator
    
        if translation is True:
        
            is_translator_enabled.append(1)
        
        else:
        
            is_translator_enabled.append(0)
            
    except:
        
        is_translator_enabled.append(0)
        
        error_objects.append(obj)
        
    try:
        
        listed_count.append(obj.user.listed_count)  # I count the number of lists that the user has
        
    except:
        
        listed_count.append(0)
        
        error_objects.append(obj)
        
    try:
    
        protection = obj.user.protected   # I check if the account of the user is protected or not
    
        if protection is True:
        
            protected.append(1)
        
        else:
        
            protected.append(0)
            
    except:
        
        protected.append(0)
        
        error_objects.append(obj)
    
    try:
    
        image=urllib.request.urlretrieve(obj.user.profile_image_url,'1.jpg')   # I download the profile picture of the user 
    
        image = face_recognition.load_image_file("1.jpg")  # I turn the picture into an object of the Face Recognition Library

        face_locations=face_recognition.face_locations(image) # I use the face recognition library to locate if there are any clear faces in the picture
    
        if face_locations==[]:
        
            image_exists.append(0)
    
        else:
        
            image_exists.append(1)
            
    except:
        
        image_exists.append(0)
        
        error_objects.append(obj)
        
    try:
        
        d = gender.Detector()
    
        name.append(obj.user.name)  # I extract the name of the user  
    
        sex = d.get_gender(str(name[i]).split()[0]) # I use the name of the user to define the user's gender
    
        if sex=='unknown':  # check if the name indicates a gender or not
        
            gender_binary.append(0)
        
        else:
        
            gender_binary.append(1)
        
        if sex is 'male' or 'mostly_male': # check if the user is male or not
        
            name_male.append(1)
        
        else:
        
            name_male.append(0)
            
    except:
        
        gender_binary.append(0)
        
        name_male.append(0)
        
        error_objects.append(obj)
        
    language=obj.user.lang # extract the language of the user    
    
    user_lang.append(language)

    screen_name.append(obj.user.screen_name)  # I extract the screen name of the user

## Creating Additional Features

In [44]:
retweet_per_second = [ retweet_count[i]/tweet_age[i] for i in range(0,len(retweet_count))]
    
favourite_per_second = [ favourite_count[i]/tweet_age[i] for i in range(0,len(tweet_age))]
    
followers_per_day = [ followers[i]/profile_age[i] for i in range(0,len(followers))]

following_per_day = [ following[i]/profile_age[i] for i in range(0,len(following))]
    
followers_following_ratio=[]

for i in range(len(followers)):
    
    try:
        
        if following[i] == 0:
        
            following[i] = 1.0
    
        a=(followers[i] / following[i])
    
        followers_following_ratio.append(a)
        
    except:
        
        followers_following_ratio.append(0)
        
        error_objects.append(obj)
        
statuses_per_day = [ statuses_count[i] / profile_age[i] for i in range(len(profile_age))]

retweet_per_follower = [ retweet_count[i] / following[i] for i in range(len(followers))]

favourite_per_follower = [ favourite_count[i] / following[i] for i in range(len(following))]

lists_per_day = [ listed_count[i] / profile_age[i] for i in range(len(profile_age))]

## Deleting the missing Tweets

In [41]:
for i in range(len(deleted_ids)):
        
    df = df[df.tweet_id != deleted_ids[i]] # I am keeping only the values that do not match with the deleted Tweets
    
df = df.reset_index(drop=True)

### Creating dataframes in order to extract the desired features

In [21]:
df2=pd.Series(list(text))

### Creating features from the text

In [22]:
characters=df2.str.len()             # I count the number of characters within a tweet.

exclamation_mark = df2.str.count('!')  # I count the number of exclamation marks.

commas = df2.str.count(',')           # I count the number of commas.

periods = df2.str.count('.')        # I count the number of periods.

question_mark = df2.str.count('\?')    # I count the number of question marks.

hashtag = df2.str.count('#')          # I count the number of hashtags.

url = df2.str.count('http')           # I count the number of urls.

mention = df2.str.count('@')          # I count the number of mentions.

words = df2.str.count(' ')+1          # I count the number of words.

### Check if there are multiple dictation marks in the Tweet

In [23]:
multiple_marks=[]

for i in range(len(exclamation_mark)):
    
    try:
    
        if exclamation_mark[i]>0 and question_mark[i]>0 : # check if the question marks and the exclamation marks are above zero
        
            multiple_marks.append(1) # in case there are write 1
        
        else:
        
            multiple_marks.append(0) # in case there are not write 0
            
    except:
        
        multiple_marks.append(0)
        
        error_objects.append(obj)

### Check how accurate is the dictation of the Tweet

In [24]:
dictation_rate=[]

for i in range(len(df2)):
    
    try:
    
        a=str(df2[i])        # define the tweet as a string variable
        
        b=TextBlob(str((df2)[i]))  # turn the tweet into a textblob object
    
        b=b.correct()   # correct the dictation of the words
        
        dictation=SequenceMatcher(None, a, b).ratio()   # compare the matching of the two strings
    
        dictation_rate.append(dictation)  # append the matching to the dictation_rate list
        
    except:
        
        dictation_rate.append(0)
        
        error_objects.append(obj)

#### Loading Files with sets of positive and negative words

In [25]:
positive = []

with open("positive-words.txt") as file:
    
    for line in file: 
        
        line = line.strip() 
        
        positive.append(line) # create a list with the positive words

In [26]:
negative = []

with open("negative-words.txt") as file:
    
    for line in file: 
        
        line = line.strip() 
        
        negative.append(line) # create a list with the negative words

### Check how many positive and negative words the Tweet entail

In [27]:
positive_words=[]

for i in range(len(df2)):
    
    try:
    
        test=(str(df2[i]))
    
        positives = [char for char in test if char in positive] # I create a list of the matching words
    
        positive_words.append(len(positives)) # I append the length of the list created
        
    except:
        
        positive_words.append(0)
        
        error_objects.append(obj)

In [28]:
negative_words=[]

for i in range(len(df2)):
    
    try:
    
        test=(str(df2[i]))
    
        negatives = [char for char in test if char in negative]   # I create a list of the matching words
    
        negative_words.append(len(negatives))   # I append the length of the list created
        
    except:
        
        negative_words.append(0)
        
        error_objects.append(obj)

### Use TextBlob Library to analyze the text of the Tweet and calculate the subjectivity and the polarity of the Tweet

In [29]:
subjectivity=[]

for i in range(len(df2)):
    
    try:
    
        test=TextBlob(str(df2[i])) # I turn the text of the tweet into a textblob object
    
        subjectivity.append(test.sentiment.subjectivity) # I append the subjectivity score to the subjectivity list
        
    except:
        
        subjectivity.append(0)
        
        error_objects.append(obj)

In [30]:
polarity=[]

for i in range(len(df2)):
    
    try:
    
        test=TextBlob(str(df2[i])) # I turn the text of the tweet into a textblob object
    
        polarity.append(test.sentiment.polarity) # I append the polarity score to the polarity list
        
    except:
        
        polarity.append(0)
        
        error_objects.append(obj)

### Count the number of uppercase letters

In [31]:
uppercase_letters=[]

for i in range(len(df2)):
    
    try:
    
        a=(str(df2[i])).split() # I break the tweet string into words
 
        count = 0

        for word in a:  
        
            for letter in word:  # With a double loop I check each letter of the string
            
                if letter.isupper():  # if a letter is uppercase
            
                    count = count + 1
            
        uppercase_letters.append(count)
        
    except:
        
        uppercase_letters.append(0)
        
        error_objects.append(obj)

### Create a list of the unique hashtags in the dataset

In [32]:
hashtag_list=[]

list_of_hashtags = []

for i in range(len(df2)):
    
    tweet_hashtags = []
    
    a=(str(df2[i])).split() # I break the tweet string into words
    
    for word in a:
    
        if word.startswith('#'):  # if the word starts with # is a hashtag
        
            tweet_hashtags.append(word)
            
            list_of_hashtags.append(word)
            
    hashtag_list.append(tweet_hashtags)
        
list_of_hashtags = list(set(list_of_hashtags))

In [33]:
with open("generic2_hastags.txt", "w") as f:
    
    for s in hashtag_list:
        
        f.write(str(s) +"\n")

### Count the number of places that are mentioned in a Tweet

In [34]:
places=[]

for i in range(len(df2)):
    
    try:
    
        a=GeoText(str(df2[i])) # I turn the string of the tweet into a Geotext object
    
        a=a.cities # I create a dictionary with the cities/places that are mentioned in the text
    
        places.append(len(a)) # I append the length of the dictionary to the places list
        
    except:
        
        places.append(0)
        
        error_objects.append(obj)

### Make a list of the unique places that are in the dataset using the GeoText library

In [35]:
places_list=[]

list_of_places = []

for i in range(len(df2)):
    
    a=GeoText(str(df2[i])) # I turn the text into an Geotext object
    
    a=a.cities
    
    places_list.append(a)
    
    for i in range(len(a)):
        
        if a[i]!=[]:  # if the tweet entails any cities/places they are appended into the places_list
            
            list_of_places.append(a[i])
        
list_of_places = list(set(list_of_places))  # I create a list with the unique places in the dataset

In [36]:
with open("generic2_places.txt", "w") as f:
    
    for s in places_list:
        
        f.write(str(s) +"\n")

### Count the number of times the First, Second and Third person pronoun is used in a Tweet

In [37]:
first_pronoun=[]

first = ['i','me','myself','mine','we','us','ours','ourselves'] # a list of the first person pronouns

for i in range(len(df2)):
    
    try:
    
        count=0
    
        a = str(df2[i]).split() # I break the tweet into words
    
        for i in range(len(a)):
        
            if a[i].lower() in first: #if a word is a first person pronoun
            
                count = count + 1
            
            first_pronoun.append(count)
            
    except:
        
        first_pronoun.append(0)
        
        error_objects.append(obj)

In [38]:
second_pronoun=[]

second = ['you','yours','yourself','yourselves']

for i in range(len(df2)):
    
    try:
    
        count=0
    
        a = str(df2[i]).split(' ')
    
        for i in range(len(a)):
        
            if a[i].lower() in second:
            
                count = count + 1
            
            second_pronoun.append(count)
            
    except:
        
        second_pronoun.append(0)
        
        error_objects.append(obj)

In [39]:
third_pronoun=[]

third = ['he','she','it','him','her','his','hers','himself','herself','itself','they','them','theirs','themselves']

for i in range(len(df2)):
    
    try:
    
        count=0
    
        a = str(df2[i]).split(' ')
    
        for i in range(len(a)):
        
            if a[i].lower() in third:
            
                count = count + 1
            
            third_pronoun.append(count)
            
    except:
        
        third_pronoun.append(0)
        
        error_objects.append(obj)

### Check for words that indicate the author of the tweet is an eye-witness of the event

In [40]:
witness_claim=[]

witness_words=['see','saw','seen', 'attend', 'attended', 'notice', 'noticed', 'watch', 'watched',
              'bystander', 'eyewitness', 'observer', 'recognize', 'recognise', 'recognized', 'recognised',
              'witness', 'witnessed', 'follow', 'followed', 'observe', 'observed']
# words that indicate that the author of the tweet is/was a witness of the event

for i in range(len(df2)):  # the process is the same as before with me counting the matching 
                            #words of the tweet to a list of words
    
    try:
    
        count=0
    
        a=str(df2[i]).split(' ')
    
        for i in range(len(a)):     
        
            if a[i].lower() in witness_words:
            
                    count = count + 1
            
            witness_claim.append(count)  
            
    except:
        
        witness_claim.append(0)
        
        error_objects.append(obj)

### Matching of user's language and the language of the Tweet

In [41]:
lang_match=[]

text_lang=[]

for i in range(len(df2)):
    
    try:
    
        a = TextBlob(str(df2[i]))
    
        text_lang.append(a.detect_language())

        for i in range(len(user_lang)): 
    
            if user_lang[i] == text_lang[i]: # check if the language of the text and the user match
        
                lang_match.append(1)
        
            else:
        
                lang_match.append(0)
            
    except:
        
        lang_match.append(0)
        
        error_objects.append(obj)

### Checking if the name of the user indicates that the user is an opinion shaper

In [42]:
opinion_shaper=[]

for i in range(len(name)):
    
    try:
    
        count=0
    
        b=str(screen_name[i]).lower()
        
        opinion=['news','breaking','news','report','report','daily','times', 'feed', 'radar', 'net']
            
        for i in range(len(opinion)):
            
            if opinion[i] in b:
        
                count = count +1
    
        if count>0:
        
            opinion_shaper.append(1)
        
        else:
        
            opinion_shaper.append(0)
            
    except:
        
        opinion_shaper.append(0)
        
        error_objects.append(obj)

## Creating a dataframe with the features created

#### Networking Features

In [45]:
metacontent_features = pd.DataFrame(list(zip(favourite_count, tweet_age, retweet_count, mention, 
                                         retweet_per_second, favourite_per_second,
                                        retweet_per_follower, favourite_per_follower)),
                            columns = ['favourite_count', 'tweet_age', 'retweet_count', 'mentions', 'retweet_per_second'
                                       ,'favourite_per_second','retweet_per_follower', 'favourite_per_follower'])

#### User profile Features

In [74]:
user_features = pd.DataFrame(list(zip(profile_age, user_description, geo_enabled, opinion_shaper,extended_profile,
                                      user_location, statuses_count, user_time_zone, verified, is_translator_enabled,
                                      listed_count, protected, notifications, default_profile_picture, image_exists, 
                                      gender_binary, name_male, statuses_per_day,lists_per_day, followers, 
                                      following, followers_per_day, following_per_day, followers_following_ratio)),
                             columns = ['profile_age', 'profile_description', 'geolocation', 'opinion_shaper',
                                        'extended_profile','user_location', 'statuses_count', 'user_timezone', 
                                        'verified_account','text_translator', 'user_lists', 'account_protection', 
                                        'notifications','default_profile_picture', 'profile_picture', 'gender', 
                                        'male', 'statuses_per_day', 'lists_per_day','followers', 
                                        'following', 'followers_per_day', 'following_per_day', 
                                        'followers_following_ratio'])

#### Tweet text features

In [80]:
linguistic_features = pd.DataFrame(list(zip(characters, exclamation_mark, question_mark, hashtag, url, words,
                                      commas, periods, multiple_marks,lang_match, dictation_rate, subjectivity,
                                        polarity, positive_words, negative_words, uppercase_letters, places, 
                                        first_pronoun, second_pronoun, third_pronoun, witness_claim)),
                             columns = ['characters', 'exclamation_marks', 'question_marks', 'hashtags', 'urls',
                                        'words', 'commas', 'periods', 'multiple_marks','lang_match', 
                                        'dictation_rate', 'subjectivity', 'polarity', 'positive_words', 
                                        'negative_words', 'uppercase_letters', 'places', 'first_person',
                                        'second_person', 'third_person', 'witness'])

### Normalizing the values of the features extracted

In [81]:
x = metacontent_features.values

y = user_features.values

z = linguistic_features.values

min_max_scaler = preprocessing.MinMaxScaler()

x_scaled = min_max_scaler.fit_transform(x)

y_scaled = min_max_scaler.fit_transform(y)

z_scaled = min_max_scaler.fit_transform(z)

metacontent_features = pd.DataFrame(x_scaled, 
                               columns = ['favourite_count', 'tweet_age', 'retweet_count', 'mentions', 
                                          'retweet_per_second', 'favourite_per_second','retweet_per_follower',
                                          'favourite_per_follower'])

user_features = pd.DataFrame(y_scaled,
                            columns = ['profile_age', 'profile_description', 'geolocation', 'opinion_shaper',
                                        'extended_profile','user_location', 'statuses_count', 'user_timezone', 
                                        'verified_account','text_translator', 'user_lists', 'account_protection', 
                                        'notifications','default_profile_picture', 'profile_picture', 'gender', 
                                        'male', 'statuses_per_day', 'lists_per_day','followers', 
                                        'following', 'followers_per_day', 'following_per_day', 
                                       'followers_following_ratio'])

linguistic_features = pd.DataFrame(z_scaled,
                            columns = ['characters', 'exclamation_marks', 'question_marks', 'hashtags', 'urls',
                                        'words', 'commas', 'periods', 'multiple_marks','lang_match', 
                                        'dictation_rate', 'subjectivity', 'polarity', 'positive_words', 'negative_words',
                                       'uppercase_letters', 'places', 'first_person', 'second_person', 'third_person', 
                                       'witness'])

In [86]:
features = pd.concat([metacontent_features,user_features,linguistic_features], axis=1)

features = pd.DataFrame(features, 
                        columns = ['favourite_count', 'tweet_age', 'retweet_count', 'mentions', 
                                          'retweet_per_second', 'favourite_per_second','retweet_per_follower',
                                          'favourite_per_follower','profile_age', 'profile_description', 
                                   'geolocation', 'opinion_shaper', 'extended_profile','user_location', 
                                   'statuses_count', 'user_timezone', 
                                        'verified_account','text_translator', 'user_lists', 'account_protection', 
                                        'notifications','default_profile_picture', 'profile_picture', 'gender', 
                                        'male', 'statuses_per_day', 'lists_per_day','followers', 
                                        'following', 'followers_per_day', 'following_per_day', 
                                       'followers_following_ratio', 'characters', 'exclamation_marks', 'question_marks', 
                                   'hashtags', 'urls', 'words', 'commas', 'periods', 'multiple_marks','lang_match', 
                                        'dictation_rate', 'subjectivity', 'polarity', 'positive_words', 'negative_words',
                                       'uppercase_letters', 'places', 'first_person', 'second_person', 'third_person', 
                                       'witness'])

,characters,exclamation_marks,question_marks,hashtags,urls,words,commas,periods,multiple_marks,lang_match,...,following,mentions,retweet_per_second,favoutire_per_second,followers_per_day,following_per_day,followers_following_ratio,retweet_per_follower,favourite_per_follower,rumor
0,0.453390,0.086957,0.000000,0.000000,0.00,0.276316,0.090909,0.453390,0.0,1.0,...,0.003558,0.0,0.0,0.000000,6.262865e-06,0.003581,5.654038e-08,0.0,0.00000,True
1,0.338983,0.043478,0.000000,0.000000,0.25,0.144737,0.000000,0.338983,0.0,0.0,...,0.002989,0.0,0.0,0.000000,2.036065e-06,0.003012,2.185529e-08,0.0,0.00000,True
2,0.457627,0.000000,0.000000,0.153846,0.25,0.184211,0.090909,0.457627,0.0,1.0,...,0.187493,0.0,0.0,0.009991,3.152742e-03,0.151742,6.716987e-07,0.0,0.00005,True
3,0.398305,0.000000,0.083333,0.000000,0.00,0.171053,0.272727,0.398305,0.0,0.0,...,0.004333,0.0,0.0,0.000000,1.956693e-06,0.004289,1.474816e-08,0.0,0.00000,True
4,0.067797,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.067797,0.0,0.0,...,0.000430,0.0,0.0,0.000000,8.209657e-07,0.000406,6.540912e-08,0.0,0.00000,True


In [87]:
features.to_csv('extracted.csv', sep='\t')

# Topic Classification Features

In [6]:
dff = pd.DataFrame(list(zip(text)), columns= ['Text'])

In [7]:
dff.to_csv('earthquake(8651_85000).csv', sep='\t')

In [13]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [14]:
dff.Text = dff.Text.apply(text_process)

In [15]:
dff.to_csv('dffprocessed.csv', sep='\t')

In [18]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(dff.Text)

In [19]:
messages_bow = bow_transformer.transform(dff.Text)